In [1]:
import re
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd

In [2]:
driver_path = './chromedriver.exe'

In [3]:
def clean_text(text):
    cleaned_text = re.sub('[a-zA-Z]', '', text)
    cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]',
                          '', cleaned_text)
    cleaned_text = ' '.join(cleaned_text.split())
    return cleaned_text

In [4]:
def biz_khan(query,driver_path,page):
    domain = '아시아투데이'
    driver = webdriver.Chrome(executable_path=driver_path)
    
    current_page = 1 #추가된 부분@@@@@@@@@@@@@@@@@@@@@@@@@
    last_page =(int(page)-1)*10+1#추가된 부분@@@@@@@@@@@@@@@@@@@@@@@@@
    
    head_list = []
    data_list = []
    
    while current_page < last_page:#추가된 부분@@@@@@@@@@@@@@@@@@@@@@@@@
#         print(current_page)
        driver = webdriver.Chrome(executable_path=driver_path)
        url_domain = 'https://search.naver.com/search.naver?where=news&query='
        news_domain ='&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=2268&nso=&is_sug_officeid=0'
        url = url_domain + query +'&%2Ca%3A&start='+str(current_page) + news_domain#추가된 부분@@@@@@@@@@@@@@@@@@@@@@@@@
#         print(url)
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')

        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
        except:
            print('기사가 없습니다.')
            driver.close()
            return 0
        driver.close()


        for url in url_list:
            driver = webdriver.Chrome(executable_path=driver_path)  # for Windows
            try:
                driver.get(url)
                time.sleep(1)
            except:
                print('link error')
                continue
            # try문 안에서 각자 언론사에 맞게 수정
            try:
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head_list.append(soup.select('#section_top > div > h3')[0].text)
                data = soup.find('div',class_='news_bm').text
                data_list.append(''.join(data))
            except:
                print(url)
                print('None_type')
            driver.close()
        current_page+=10#추가된 부분@@@@@@@@@@@@@@@@@@@@@@@@@
        
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                          '제목' :  head_list,
                          '내용' : data_list})
    return output

biz = biz_khan('풀무원 바른먹거리',driver_path,5)#추가된 부분@@@@@@@@@@@@@@@@@@@@@@@@@
biz

,언론사,제목,내용
0,아시아투데이,"풀무원, ‘바른먹거리 캠페인’ 전국 확대…2만여명에 무상 교육",[사진1] 풀무원 바른먹거리 캠페인 교육_미취학아동 미각교육0풀무원 바른먹거리 캠페...
1,아시아투데이,"풀무원다논, 무주 공장 요거트 생산라인 증설…239억원 투입",풀무원다논0전북 무주농공단지에 위치한 풀무원다논 요거트 공장 전경/제공 = 풀무원다...
2,아시아투데이,"풀무원식품, 신임 대표에 김진홍 전 LG전자 글로벌마케팅센터장 선임",[사진] 풀무원식품(주) 신임 김진홍 대표 (1)0김진홍 풀무원식품 신임 대표이사 ...
3,아시아투데이,"풀무원, 어린이 2만2000명에 ‘바른먹거리 캠페인 교육’ 실시",무0사진=풀무원 풀무원은 어린이에게 바른 식습관을 길러주는 ‘바른먹거리 캠페인 교육...
4,아시아투데이,"풀무원, ‘대한민국소셜미디어대상’ 종합식품·소셜브랜드부문 대상",ㄴㄴ0풀무원은 15일 서울 중구 한국프레스센터에서 열린 ‘2017 대한민국 소셜미디...
5,아시아투데이,"풀무원, 어른들을 위한 ‘성인 바른먹거리 교육’ 실시…“어른이 먼저 불균형 식습관 ...",풀무원0성인 바른먹거리 교육 시간에 강사가 교육생들에게 당 흡수를 줄이는 식생활 지...
6,아시아투데이,풀무원 ‘바른먹거리 교육’ 영역 넓힌다,[사진3] 요리실습_멸치주먹밥 만들기10풀무원의 ‘바른먹거리 캠페인’에 참가한 어린...
7,아시아투데이,"풀무원녹즙, 3중 필터링 거친 노니주스 ‘노니엔클리어’ 출시","[사진자료] 풀무원녹즙, 노니엔클리어 제품 이미지0 풀무원의 신선음료 브랜드 풀무원..."
8,아시아투데이,"풀무원, ‘바른먹거리’ 교육 확대…올해 1만명 실시",바른먹거리_교육_사진10풀무원이 주최한 ‘바른먹거리 캠페인’ 교육에서 어린이들이 식...
9,아시아투데이,"“용기째 요리”…풀무원, 용기형 가정간편식 ‘5분 키친 순두부 요리’ 3종 출시",사진10 풀무원이 간편하고 손쉽게 순두부 요리를 즐길 수 있는 용기형 가정간편식 5...


In [5]:
biz.to_csv('아시아투데이.csv',encoding='utf-8-sig',index=None)